In [1]:
# Code to be able to import local modules in notebooks
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:

# Import dependencies
# Standard python libraries
import os

from oauthlib.oauth2 import WebApplicationClient
import requests
import pandas as pd

# Internal imports
from ffpackage.scraping import mfl
from ffpackage.analysis import analysis
# from ffpackage.scraping import ffdb
# from ffpackage.scraping import ourlads
#from ffpackage.viz import viz


from appmanager.database import db
from appmanager.user import user




# Configuration (These variables are stored as environment variables)
GOOGLE_CLIENT_ID = os.environ.get("GOOGLE_CLIENT_ID", None)
GOOGLE_CLIENT_SECRET = os.environ.get("GOOGLE_CLIENT_SECRET", None)
GOOGLE_DISCOVERY_URL = ("https://accounts.google.com/.well-known/openid-configuration")

In [3]:
#def getFranchise():
user_league = "53906"

franchises = mfl.get_franchises(user_league)['franchiseName'].to_list()

franchises

['Providence Monastic Spirits',
 'Washington Fantasy Football Team',
 'Verdansk Vaqueros FFC',
 'Down with the Sickness unto Death',
 'Crocodilopolis Body Snatchers',
 "O'Henry's Stiffies",
 'Comeback Kings',
 'WHAM! BAM! Thank You CAM!',
 'Pretty Big Wieners',
 'Saskatoon Squatches',
 'Gus The Bus',
 'IDK much about soccer ']

In [4]:
#def allPlayers():
player_df = db.get_df("player_df")
tables = [player_df.to_html(classes='data')]
titles=player_df.columns.values
#return render_template("allPlayers.html", tables=tables, titles=titles)

/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [5]:
#@app.route('/waiverWire', methods=['GET', 'POST'])
#@login_required
#def waiverWire():
user_league = "53906"
user_franchise = "0005"

# Get Franchises in the league
franchise_df = mfl.get_franchises(user_league)
# Append a row to carry free agents
franchise_df = franchise_df.append({"franchiseID":"FA", "franchiseName":"Free Agent"}, ignore_index=True)

# Get franchise rosters
rosters_df = mfl.get_rosters(user_league, user_franchise=user_franchise)

# Get Free Agents
freeAgent_df = mfl.get_freeAgents(user_league)

# Combine Franchise rosters with free agents to get all players
rosters_df = pd.concat([rosters_df, freeAgent_df], axis=0)

# Get all players, sharkRank, and ADP
player_df = db.get_df("predictions")

# Merge all dfs
complete = player_df.merge(rosters_df, on='id_mfl', how='left').merge(franchise_df[['franchiseID', 'franchiseName']], on='franchiseID', how='left')
complete = complete[complete['franchiseID'].notna()]
complete = complete.sort_values(by=['pred'], ascending=False)
complete.reset_index(inplace=True, drop=True)
complete = complete[['player', 'age', 'team', 'franchiseName', 'pos', 'posRank', 'KR', 'PR', 'RES', 'pred', 'sharkAbsolute', 'adpAbsolute']]
complete.rename(columns={
    'player':'Player',
    'age':'Age',
    'team':'Team',
    'pos':'Position',
    'posRank': 'Rank',
    'pred': 'My Prediction',
    'sharkAbsolute': 'FantasySharks Prediction',
    'adpAbsolute': 'ADP-Based Prediction'
}, inplace=True)
complete.set_index('Player', drop=True, inplace=True)

#return render_template("waiverWire.html", tables=[complete.to_html(classes='data')], titles=complete.columns.values)

/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_25223/2224525526.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  franchise_df = franchise_df.append({"franchiseID":"FA", "franchiseName":"Free Agent"}, ignore_index=True)
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [6]:
#@app.route('/compareFranchises2')
#@login_required
#def compareFranchises2():
user_league = "53906"

# Get Franchises in the league
franchise_df = mfl.get_franchises(user_league)
franchise_df = franchise_df.append({"FranchiseID":"FA", "FranchiseName":"Free Agent"}, ignore_index=True)

# Get franchise rosters
rosters_df = mfl.get_rosters(user_league)

# Get Free Agents
freeAgent_df = mfl.get_freeAgents(user_league)

# Get all players, sharkRank, and ADP
predictions = db.get_df("predictions")

# Merge all dfs
complete = predictions.merge(rosters_df, on='id_mfl', how='left').merge(franchise_df[['franchiseID', 'franchiseName']], on='franchiseID', how='left')
complete['franchiseID'].fillna("FA", inplace=True)
complete['franchiseName'].fillna("Free Agent", inplace=True)
complete['rosterStatus'].fillna("Free Agent", inplace=True)
complete


/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_25223/2371207576.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  franchise_df = franchise_df.append({"FranchiseID":"FA", "FranchiseName":"Free Agent"}, ignore_index=True)
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


,id_mfl,player,age,team,pos,posRank,KR,PR,RES,sharkRank,adp,pred,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute,franchiseID,week,rosterStatus,franchiseName
0,5848,TOM BRADY,45.0,TBB,QB,QB1,NO,NO,NO,105.0,167.0,347.798672,118.080001,275.680001,47.610000,205.210000,0002,7,ROSTER,Washington Fantasy Football Team
1,14779,JUSTIN HERBERT,24.0,LAC,QB,QB1,NO,NO,NO,66.0,34.0,341.323461,171.646664,329.246664,178.046668,335.646667,0004,7,ROSTER,Down with the Sickness unto Death
2,13116,PATRICK MAHOMES,27.0,KCC,QB,QB1,NO,NO,NO,57.0,36.6,339.347453,193.409999,351.009999,171.646664,329.246664,0010,7,ROSTER,Saskatoon Squatches
3,12620,DAK PRESCOTT,29.0,DAL,QB,QB1,NO,NO,NO,930.0,117.4,332.541423,5.396669,162.996669,118.080001,275.680001,0001,7,ROSTER,Providence Monastic Spirits
4,9431,MATTHEW STAFFORD,34.0,LAR,QB,QB1,NO,NO,NO,119.0,129.6,331.415001,85.600002,243.200002,61.706666,219.306665,0007,7,ROSTER,Comeback Kings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,14838,LAVISKA SHENAULT,24.0,CAR,WR,WR3,NO,NO,NO,821.0,3000.0,24.121850,-49.436667,66.163333,-108.266667,7.333333,FA,NaN,Free Agent,Free Agent
666,14555,GUNNER OLSZEWSKI,25.0,PIT,WR,WR2,KR1,PR1,NO,1107.0,3000.0,107.820542,-94.263334,105.336666,-15.186667,184.413333,FA,NaN,Free Agent,Free Agent
667,12394,DEANDRE CARTER,29.0,LAC,WR,WR2,KR1,PR1,NO,676.0,3000.0,105.890985,-41.863333,157.736667,-84.720000,114.880000,0010,7,ROSTER,Saskatoon Squatches
668,14108,GREG DORTCH,24.0,ARI,WR,WR2,NO,PR1,NO,891.0,3000.0,46.935179,-54.880000,86.220000,-111.460000,29.640000,0006,7,ROSTER,O'Henry's Stiffies


In [8]:
# Get info on available slots from MFL site
posmax = {"QB":2, "RB":5, "WR":6, "TE":5, "PK":2, "DF":2}
posmin = {"QB":1, "RB":2, "WR":2, "TE":2, "PK":2, "DF":2}
totalStarters = 15
predMethod = "pred"

df = analysis.starterSelector(complete, predMethod, totalStarters, posmax, posmin)
df



,id_mfl,player,age,team,pos,posRank,KR,PR,RES,sharkRank,...,sharkRelative,sharkAbsolute,adpRelative,adpAbsolute,franchiseID,week,rosterStatus,franchiseName,rosterRank,starting
0,5848,TOM BRADY,45.0,TBB,QB,QB1,NO,NO,NO,105.0,...,118.080001,275.680001,47.610000,205.210000,0002,7,ROSTER,Washington Fantasy Football Team,1.0,Starter
1,14779,JUSTIN HERBERT,24.0,LAC,QB,QB1,NO,NO,NO,66.0,...,171.646664,329.246664,178.046668,335.646667,0004,7,ROSTER,Down with the Sickness unto Death,1.0,Starter
2,13116,PATRICK MAHOMES,27.0,KCC,QB,QB1,NO,NO,NO,57.0,...,193.409999,351.009999,171.646664,329.246664,0010,7,ROSTER,Saskatoon Squatches,1.0,Starter
3,12620,DAK PRESCOTT,29.0,DAL,QB,QB1,NO,NO,NO,930.0,...,5.396669,162.996669,118.080001,275.680001,0001,7,ROSTER,Providence Monastic Spirits,1.0,Starter
4,9431,MATTHEW STAFFORD,34.0,LAR,QB,QB1,NO,NO,NO,119.0,...,85.600002,243.200002,61.706666,219.306665,0007,7,ROSTER,Comeback Kings,1.0,Starter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,14838,LAVISKA SHENAULT,24.0,CAR,WR,WR3,NO,NO,NO,821.0,...,-49.436667,66.163333,-108.266667,7.333333,FA,NaN,Free Agent,Free Agent,NaN,Bench
666,14555,GUNNER OLSZEWSKI,25.0,PIT,WR,WR2,KR1,PR1,NO,1107.0,...,-94.263334,105.336666,-15.186667,184.413333,FA,NaN,Free Agent,Free Agent,NaN,Bench
667,12394,DEANDRE CARTER,29.0,LAC,WR,WR2,KR1,PR1,NO,676.0,...,-41.863333,157.736667,-84.720000,114.880000,0010,7,ROSTER,Saskatoon Squatches,7.0,Bench
668,14108,GREG DORTCH,24.0,ARI,WR,WR2,NO,PR1,NO,891.0,...,-54.880000,86.220000,-111.460000,29.640000,0006,7,ROSTER,O'Henry's Stiffies,6.0,Bench


In [22]:

# Find the lowest scoring player on the field and set them as the low bar
for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
    positionMinPred = df.loc[(df['pos']==x) & (df['starting']=='Starter'), predMethod].min()
    # Calculate relative values
    df.loc[df['pos']==x, 'relativeValue'] = df.loc[df['pos']==x, predMethod] - positionMinPred


KeyError: 'FranchiseName'

In [ ]:

# Create bar chart
figADP = px.bar(players_onthefield, 
            x="FranchiseName", 
            y="adpRelative", 
            color="pos", 
            text='player', 
            color_discrete_map={
                "QB": "hsla(210, 60%, 25%, 1)", #blue #1033a6 #0c2987 1033a6 062647 #293745 
                "RB": "hsla(12, 50%, 45%, 1)", #gold #f5d000 ffa524 a23419 a34e39
                "WR": "hsla(267, 40%, 45%, 1)", #purple #4f22bc #643fc1 643fc1 621B74 675280
                "TE": "hsla(177, 68%, 36%, 1)", #teal #02687b #038097 1295ad 43B3AE
                "PK": "hsla(14, 30%, 40%, 1)", #gold #f5d000 ffa524 664e47
                "DF": "hsla(35, 70%, 65%, 1)"}, #gold #f5d000 ffa524 a49375 ffb54d
            category_orders={
                "pos": ["QB", "RB", "WR", "TE", "PK", "DF"]},
            hover_name="player",
            hover_data={
                'adpRelative':True, 'pred':True, 'sharkAbsolute':True, 'adpAbsolute':True,
                'player':False, 'pos':False, 'FranchiseName':False
                },
            labels={
                "FranchiseName":"Franchise",
                "adpRelative":"Player Value",
                "pred":"ChopBlock Prediction",
                "sharkAbsolute":"FantasySharks Prediction",
                "adpAbsolute":"ADP-Based Prediction"
            }
            )
figADP.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="ADP-Based Predictions",
            font_family="Skia",
            showlegend=False
            )

graphJSON_adp = json.dumps(figADP, cls=plotly.utils.PlotlyJSONEncoder)


### Shark Predictions
# Roster Builder logic
qbs_top = qbs.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(1)
rbs_top = rbs.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
wrs_top = wrs.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)
tes_top = tes.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
pks_top = pks.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
defs_top = defs.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)

qbs_remainder = qbs[~qbs['PlayerID'].isin(qbs_top['PlayerID'])].groupby('FranchiseName').head(1)
rbs_remainder = rbs[~rbs['PlayerID'].isin(rbs_top['PlayerID'])].groupby('FranchiseName').head(3)
wrs_remainder = wrs[~wrs['PlayerID'].isin(wrs_top['PlayerID'])].groupby('FranchiseName').head(3)
tes_remainder = tes[~tes['PlayerID'].isin(tes_top['PlayerID'])].groupby('FranchiseName').head(3)

remainder = pd.concat([qbs_remainder, rbs_remainder, wrs_remainder, tes_remainder])

top_remainders = remainder.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)

players_onthefield = pd.concat([qbs_top, rbs_top, wrs_top, tes_top, pks_top, defs_top, top_remainders])
players_onthefield = players_onthefield.sort_values(by='sharkAbsolute', ascending=False, ignore_index=True)

fran_rank = players_onthefield.groupby('FranchiseName').sum().sort_values(by='sharkAbsolute', ascending=False)

sorter = fran_rank.index

players_onthefield.FranchiseName = players_onthefield.FranchiseName.astype("category")
players_onthefield.FranchiseName.cat.set_categories(sorter, inplace=True)
players_onthefield.sort_values(["FranchiseName"], inplace=True)

# remove Free Agents
players_onthefield = players_onthefield.loc[players_onthefield.FranchiseID!="FA"]

# Find the lowest scoring player on the field and set them as the low bar
for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
    players_onthefield.loc[players_onthefield['pos']==x, 'sharkComp'] = players_onthefield.loc[players_onthefield['pos']==x, 'sharkAbsolute'].min()
players_onthefield['sharkRelative'] = players_onthefield['sharkAbsolute'] - players_onthefield['sharkComp']

# Create bar chart
figShark = px.bar(players_onthefield, 
            x="FranchiseName", 
            y="sharkRelative", 
            color="pos", 
            text='player', 
            color_discrete_map={
                "QB": "hsla(210, 60%, 25%, 1)", #blue #1033a6 #0c2987 1033a6 062647 #293745 
                "RB": "hsla(12, 50%, 45%, 1)", #gold #f5d000 ffa524 a23419 a34e39
                "WR": "hsla(267, 40%, 45%, 1)", #purple #4f22bc #643fc1 643fc1 621B74 675280
                "TE": "hsla(177, 68%, 36%, 1)", #teal #02687b #038097 1295ad 43B3AE
                "PK": "hsla(14, 30%, 40%, 1)", #gold #f5d000 ffa524 664e47
                "DF": "hsla(35, 70%, 65%, 1)"}, #gold #f5d000 ffa524 a49375 ffb54d
            category_orders={
                "pos": ["QB", "RB", "WR", "TE", "PK", "DF"]},
            hover_name="player",
            hover_data={
                'sharkRelative':True, 'pred':True, 'sharkAbsolute':True, 'adpAbsolute':True,
                'player':False, 'pos':False, 'FranchiseName':False
                },
            labels={
                "FranchiseName":"Franchise",
                "sharkRelative":"Player Value",
                "pred":"ChopBlock Prediction",
                "sharkAbsolute":"FantasySharks Prediction",
                "adpAbsolute":"ADP-Based Prediction"
            }
            )
figShark.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="FantasySharks Predictions",
            font_family="Skia",
            showlegend=False
            )

graphJSON_shark = json.dumps(figShark, cls=plotly.utils.PlotlyJSONEncoder)



### My predictions
# Roster Builder logic
qbs_top = qbs.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(1)
rbs_top = rbs.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
wrs_top = wrs.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)
tes_top = tes.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
pks_top = pks.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)
defs_top = defs.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(2)

qbs_remainder = qbs[~qbs['PlayerID'].isin(qbs_top['PlayerID'])].groupby('FranchiseName').head(1)
rbs_remainder = rbs[~rbs['PlayerID'].isin(rbs_top['PlayerID'])].groupby('FranchiseName').head(3)
wrs_remainder = wrs[~wrs['PlayerID'].isin(wrs_top['PlayerID'])].groupby('FranchiseName').head(3)
tes_remainder = tes[~tes['PlayerID'].isin(tes_top['PlayerID'])].groupby('FranchiseName').head(3)

remainder = pd.concat([qbs_remainder, rbs_remainder, wrs_remainder, tes_remainder])

top_remainders = remainder.sort_values(by='pred', ascending=False, ignore_index=True).groupby('FranchiseName').head(3)

players_onthefield = pd.concat([qbs_top, rbs_top, wrs_top, tes_top, pks_top, defs_top, top_remainders])
players_onthefield = players_onthefield.sort_values(by='pred', ascending=False, ignore_index=True)

fran_rank = players_onthefield.groupby('FranchiseName').sum().sort_values(by='pred', ascending=False)

sorter = fran_rank.index

players_onthefield.FranchiseName = players_onthefield.FranchiseName.astype("category")
players_onthefield.FranchiseName.cat.set_categories(sorter, inplace=True)
players_onthefield.sort_values(["FranchiseName"], inplace=True)

# remove Free Agents
players_onthefield = players_onthefield.loc[players_onthefield.FranchiseID!="FA"]

# Find the lowest scoring player on the field and set them as the low bar
for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
    players_onthefield.loc[players_onthefield['pos']==x, 'predComp'] = players_onthefield.loc[players_onthefield['pos']==x, 'pred'].min()
players_onthefield['predRelative'] = players_onthefield['pred'] - players_onthefield['predComp']

# Create bar chart
figPred = px.bar(players_onthefield, 
            x="FranchiseName", 
            y="predRelative", 
            color="pos", 
            text='player', 
            color_discrete_map={
                "QB": "hsla(210, 60%, 25%, 1)", #blue #1033a6 #0c2987 1033a6 062647 #293745 
                "RB": "hsla(12, 50%, 45%, 1)", #gold #f5d000 ffa524 a23419 a34e39
                "WR": "hsla(267, 40%, 45%, 1)", #purple #4f22bc #643fc1 643fc1 621B74 675280
                "TE": "hsla(177, 68%, 36%, 1)", #teal #02687b #038097 1295ad 43B3AE
                "PK": "hsla(14, 30%, 40%, 1)", #gold #f5d000 ffa524 664e47
                "DF": "hsla(35, 70%, 65%, 1)"}, #gold #f5d000 ffa524 a49375 ffb54d
            category_orders={
                "pos": ["QB", "RB", "WR", "TE", "PK", "DF"]},
            hover_name="player",
            hover_data={
                'predRelative':True, 'pred':True, 'sharkAbsolute':True, 'adpAbsolute':True,
                'player':False, 'pos':False, 'FranchiseName':False
                },
            labels={
                "FranchiseName":"Franchise",
                "predRelative":"Player Value",
                "pred":"ChopBlock Prediction",
                "sharkAbsolute":"FantasySharks Prediction",
                "adpAbsolute":"ADP-Based Prediction"
            }
            )
figPred.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="ChopBlock Predictions",
            font_family="Skia",
            showlegend=False
            )

graphJSON_pred = json.dumps(figPred, cls=plotly.utils.PlotlyJSONEncoder)
#return render_template('compareFranchises2.html', graphJSON_pred=graphJSON_pred, graphJSON_adp=graphJSON_adp, graphJSON_shark=graphJSON_shark)
